In [10]:
import os
import gc
import pyfesom2 as pf
import xarray as xr
import numpy as np
import shutil
import random
import string
from scipy.spatial import cKDTree
import dask
from dask import delayed
import dask.array as da
from dask.distributed import Client,wait,Lock
dask.config.config.get('distributed').get('dashboard').update({'link':'{JUPYTERHUB_SERVICE_PREFIX}proxy/{port}/status'})

In [2]:
year=2015
time_chunk = 1; #chunk data saily -> each day is treated separately anyway
nod2_chunk = 11538465; #make sure no chunking in space!

In [3]:
# paths
data_path = '/p/scratch/chhb19/mueller29/AO_40_nopc/'
mesh_path = '/p/project/chhb19/meshes/AO_40/'
composite_path = '/p/scratch/chhb19/mueller29/composites/100m/'
# load fesom mesh stuff
mesh=pf.load_mesh(mesh_path)
model_lons=mesh.x2
model_lats=mesh.y2

# rotate to equator (eddy detection stuff is saved in rotated coordinates)
lons_rot,lats_rot = pf.ut.scalar_g2r(-90,90,90,model_lons,model_lats)

/p/project/chhb19/meshes/AO_40/pickle_mesh_py3_fesom2
The usepickle == True)
The pickle file for FESOM2 exists.
The mesh will be loaded from /p/project/chhb19/meshes/AO_40/pickle_mesh_py3_fesom2


In [4]:
def generate_random_string(length):
    # Define the characters to choose from
    characters = string.ascii_letters + string.digits  # You can add more characters if needed
    # Use random.choices to generate a list of random characters
    random_characters = random.choices(characters, k=length)
    # Join the characters into a string
    random_string = ''.join(random_characters)
    return random_string
#spawn a parallel cluster
n_cores = 10
mem_lim = str(int(100*np.floor(960/n_cores)))+'MB' #96GB total memory, set to MB (96000), divide by number of cores, then round to next 100
dask_dir = '/p/scratch/chhb19/mueller29/dask_dir/'+generate_random_string(10)
if os.path.exists(dask_dir):
    shutil.rmtree(dask_dir)
if 'client' in locals() or 'client' in globals():
    client.close()
client = Client(local_directory=dask_dir,n_workers=n_cores, threads_per_worker=1,memory_limit=mem_lim)
client.amm.start()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/vasco.mueller_at_awi.de/r0b111d10d9c453583df569be68e9959/proxy/8787/status,
Dashboard: /user/vasco.mueller_at_awi.de/r0b111d10d9c453583df569be68e9959/proxy/8787/status,Workers: 10
Total threads: 10,Total memory: 89.41 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:44603,Workers: 10
Dashboard: /user/vasco.mueller_at_awi.de/r0b111d10d9c453583df569be68e9959/proxy/8787/status,Total threads: 10
Started: Just now,Total memory: 89.41 GiB
Comm: tcp://127.0.0.1:41323,Total threads: 1
Dashboard: /user/vasco.mueller_at_awi.de/r0b111d10d9c453583df569be68e9959/proxy/40635/status,Memory: 8.94 GiB
Nanny: tcp://127.0.0.1:40323,


In [5]:
a_ice = xr.open_mfdataset(data_path+'a_ice.fesom.*.nc',chunks={'time':time_chunk,'nod2':nod2_chunk})['a_ice'].astype('float32')
dates = a_ice.time

In [6]:
def lon_lat_to_cartesian(lon, lat, R=6371000):
    """
    calculates lon, lat coordinates of a point on a sphere with
    radius R. Taken from http://earthpy.org/interpolation_between_grids_with_ckdtree.html
    """
    lon_r = np.radians(lon)
    lat_r = np.radians(lat)

    x = R * np.cos(lat_r) * np.cos(lon_r)
    y = R * np.cos(lat_r) * np.sin(lon_r)
    z = R * np.sin(lat_r)
    return x, y, z

def create_indexes_and_distances(model_lon, model_lat, lons, lats, k=1, workers=2):
    """
    Creates KDTree object and query it for indexes of points in FESOM mesh that are close to the
    points of the target grid. Also return distances of the original points to target points.
    Parameters
    """
 
    xs, ys, zs = lon_lat_to_cartesian(model_lon, model_lat)
    xt, yt, zt = lon_lat_to_cartesian(lons.flatten(), lats.flatten())

    tree = cKDTree(list(zip(xs, ys, zs)))
    distances, inds = tree.query(list(zip(xt, yt, zt)), k=k, workers=workers)

    return distances, inds

@delayed
def process_date(date, a_ice_tmp, lons_rot, lats_rot,lock,n_neighbors,n_cores):
    day = date.dt.day.values
    month = date.dt.month.values
    year = date.dt.year.values
    date_string = ''.join([str(year), str(month).zfill(2), str(day).zfill(2)])

    composite_file = (composite_path+'/a_ice/composite_a_ice_'+date_string+'.nc')
    with xr.open_dataset(composite_file, mode='a') as ds_composite:
        eddy_lats=ds_composite['lat'].astype('float32')[0,:].values
        eddy_lons=ds_composite['lon'].astype('float32')[0,:].values
    ds_composite.close()
    outfiles=[]
    vars=['a_ice','m_ice','fh','temp_100']
    for var in vars:
        outfiles.append(composite_path+var+'/composite_'+var+'_'+date_string+'.nc')
    _,inds = create_indexes_and_distances(lons_rot,lats_rot,eddy_lons,eddy_lats,n_neighbors,n_cores)
    # Convert inds into an xarray.DataArray
    inds = xr.DataArray(inds, dims=('ee', 'tmp'))
    a_ice_avg = a_ice_tmp.isel(nod2=inds).mean(dim='tmp').expand_dims(dim='time').compute()
    for outfile in outfiles:  
        #with lock:
        with xr.open_dataset(outfile, mode='a') as ds:
            ds['ice_avg'] = a_ice_avg
            ds.to_netcdf(outfile, mode='a', encoding={'ice_avg': {'dtype': 'float32'}})
        ds.close()
    del a_ice_tmp
    del lons_rot
    del lats_rot
    del eddy_lats
    del eddy_lons
    del a_ice_avg
    del inds

    

In [7]:
n_neighbors=3


In [ ]:
batch_size = 20  # Adjust the batch size as needed
results = []
lock = Lock()
# Iterate over the dates in batches
for batch_start in range(0, len(dates), batch_size):
    batch_dates = dates[batch_start:batch_start + batch_size]

    # Compute results for the current batch
    batch_results = []
    client.persist(a_ice)
    for dd, date in enumerate(batch_dates):
        a_ice_tmp = a_ice[dd, :]
        client.persist(a_ice_tmp)
        batch_results.append(process_date(date, a_ice_tmp, lons_rot, lats_rot, lock, n_neighbors, n_cores))

    # Execute the delayed computations for the current batch
    _ = dask.compute(*batch_results)
    # Collect garbage after computing each batch
    gc.collect()

In [13]:
client.close()